# Parametrize the volume of slices

Author: [Chiara Meroni](https://merochia.wixsite.com/chiara-meroni/)

updated to v1.0 by Michael Joswig

Based on: Berlow, Brandenburg, Meroni, Shankar. [Intersection bodies of polytopes](https://doi.org/10.1007/s13366-022-00621-7). Beiträge zur Algebra und Geometrie, 63, 419-439 (2022).

In this notebook we want to compute the volume of linear hyperplane sections of polytopes parametrically. More precisely, fix a polytope $P\subset \mathbb{R}^3$. In the example we have an Archimedean solid: the truncated tetrahedron $P$. Given $x \in \mathbb{R}^3$, we compute the function
$$
\rho(x) = \frac{1}{\|x\|} \operatorname{vol}( P \cap x^\perp)
$$
where $x^\perp$ is the hyperplane through the origin orthogonal to $x$. This function is known in convex geometry as the $\textit{radial function}$ of the intersection body of $P$. It turns out that $\rho$ is a piecewise rational function, and the pieces are polyhedral cones.

In [1]:
using Oscar

  ___   ____   ____    _    ____
 / _ \ / ___| / ___|  / \  |  _ \   |  Combining ANTIC, GAP, Polymake, Singular
| | | |\___ \| |     / _ \ | |_) |  |  Type "?Oscar" for more information
| |_| | ___) | |___ / ___ \|  _ <   |  Manual: https://docs.oscar-system.org
 \___/ |____/ \____/_/   \_\_| \_\  |  Version 1.0.0


In [2]:
P = archimedean_solid("truncated_tetrahedron")

Polyhedron in ambient dimension 3

Compute the associated zonotope $Z(P)$ which is the Minkowski sum of $[-v,v]$ for the vertices $v$ of $P$. The normal fan $\Sigma$ of $Z(P)$ defines the regions in which the function $\rho$ is rational.

In [3]:
Z = sum(convex_hull([-v, v]) for v in vertices(P))
Σ = normal_fan(Z)

Polyhedral fan in ambient dimension 3

In each such maximal cone, the combinatorial type of the section $P\cap x^\perp$ does not change, when $x$ varies in a given maximal cone. Our goal is to compute $\rho$ cone by cone.

We start by constructing one point in the interior of each cone, and we collect them in `u_samples`. This specific points are going to determine the combinatorial structure of $P\cap x^\perp$ in each cone.

In [4]:
u_samples = [sum(rays(C)) for C in maximal_cones(Σ)]

134-element Vector{RayVector{QQFieldElem}}:
 [-4, -9//2, -27//2]
 [-2, -4, -12]
 [-3, -3//2, -19//2]
 [4, 9//2, -27//2]
 [3, 3//2, -19//2]
 [2, 4, -12]
 [0, 0, -28]
 [-4, -27//2, -9//2]
 [-4, -11, -11]
 [0, -6, -14]
 [2, -4, -12]
 [4, 27//20, -27//20]
 [5, 11//5, -32//5]
 ⋮
 [-4, -27//20, 27//20]
 [-2, 4, 12]
 [0, 6, 14]
 [4, 11, 11]
 [4, 27//2, 9//2]
 [0, 0, 28]
 [-2, -4, 12]
 [-3, -3//2, 19//2]
 [-4, -9//2, 27//2]
 [3, 3//2, 19//2]
 [2, 4, 12]
 [4, 9//2, 27//2]

For the sake of simplicity, we analyze here the steps to compute the radial function $\rho$ in the first maximal cone. The same procedure should be applied to every cone of $\Sigma$ in order to obtain the complete radial function of the intersection body.

The vertices of the hyperplane section $P\cap x^\perp$ arise as the intersection of the hyperplane with edges of $P$.
Therefore, we collect in `good_edges` the edges $e$ of $P$ such that $\text{dim}(e\cap x^{\perp})=0$. These are the edges whose extrema lie in distict halfspaces defined by $x^\perp$. We identify an edge $e$ with a list $[i,j]$ meaning that the extrema of $e$ are the $i$-th and the $j$-th vertices of the polytope $P$.

In [5]:
# choose here one specific cone for simplicity
u_sample = u_samples[1]

# collect the edges intersected by the hyperplane orthogonal to u_sample
good_edges = []
for edge in edges(edgegraph(P))
    v1 = src(edge)
    v2 = dst(edge)
    first = transpose(Vector(u_sample)) * Vector(vertices(P)[v1])
    last = transpose(Vector(u_sample)) * Vector(vertices(P)[v2])
    if first * last < 0
        push!(good_edges, [v1, v2])
    end
end
println(good_edges)

Any[[9, 7], [9, 8], [10, 3], [11, 6]]


We want to compute now the volume of the section via a triangulation: the volume of $P\cap x^\perp$ is the sum of the volumes of the simplices in its triangulation. We require that each simplex has the origin as one of its vertices.
Since the hyperplane section is a polygon, the simplices of the triangulation are the convex hull of an edge of $P\cap x^\perp$ and the origin. 

Therefore, we collect in `good_facet_edges` the pairs of edges of $P$ that belong to a common facet; they provide vertices of $P\cap x^\perp$ connected by an edge. Again, we identify an edge with the indices of its vertices.

In [6]:
good_facet_edges = []
vertex_facet_incidences = vertex_indices(facets(P))
nrows, _ = size(vertex_facet_incidences)

for i in 1:nrows
    facet_as_set = Set(findall(vertex_facet_incidences[i,:]))
    intersecting_edges = []
    for edge in good_edges
        test = length(intersect(Set(edge),facet_as_set))
        if test == 2
            push!(intersecting_edges, edge)
        end
    end
    if length(intersecting_edges) >= 2
        push!(good_facet_edges, intersecting_edges)
    end
end
print(good_facet_edges)

Any[Any[[9, 8], [11, 6]], Any[[9, 7], [10, 3]], Any[[10, 3], [11, 6]], Any[[9, 7], [9, 8]]]

A vertex $v$ of $P\cap x^\perp$ belonging to the edge with extrema $a,b$ can be written as $\frac{b⋅x}{(b-a)⋅x}(a - b)+b$. For every pair of edges in `good_facet_edges`, we construct an associated matrix $M$, whose rows are the two vertices of $P\cap x^\perp$ identified by the given pair of edges, and the vector $(x_1,\ldots ,x_3)$. 
The determinant of $M$ is, up to sign and a factor of $\frac{1}{2\|x\|}$, the volume of the simplex in the triangulation corresponding to this pair of edges of $P$.

In order to compute these determinants, we use the fraction field of the polynomial ring $\mathbb{Q}[x,y,z]$.

In [7]:
R, (x, y, z) = polynomial_ring(QQ, ["x", "y", "z"])
S = fraction_field(R) 

Fraction field
  of multivariate polynomial ring in 3 variables over QQ

In [8]:
var = S[x y z]

[x   y   z]

We collect the vertices of $P\cap x^\perp$ in `Q_vertices`, keeping track of the structure of the hyperplane section, captured in `good_facet_edges`.

In [9]:
vert = [[[vertices(P)[e[1]],vertices(P)[e[2]]] for e in edge_list] for edge_list in good_facet_edges]
vert = [[[[S.(j) for j ∈ entries] for entries ∈ v] for v ∈ vv] for vv ∈ vert]
Q_vertices = [[((var*v[2])[1] //(var*(v[2]-v[1]))[1]).*(v[1]-v[2]) .+ v[2]  for v ∈ vv] for vv ∈ vert]

4-element Vector{Vector{Vector{AbstractAlgebra.Generic.FracFieldElem{QQMPolyRingElem}}}}:
 [[(1//3*y - 4//3*z)//(x + z), -1//3, (4//3*x + 1//3*y)//(x + z)], [-y//(x + z), 1, -y//(x + z)]]
 [[-1//3, (1//3*x - 4//3*z)//(y + z), (1//3*x + 4//3*y)//(y + z)], [1, -x//(y + z), -x//(y + z)]]
 [[1, -x//(y + z), -x//(y + z)], [-y//(x + z), 1, -y//(x + z)]]
 [[-1//3, (1//3*x - 4//3*z)//(y + z), (1//3*x + 4//3*y)//(y + z)], [(1//3*y - 4//3*z)//(x + z), -1//3, (4//3*x + 1//3*y)//(x + z)]]

Finally we compute the determinants of the simplices of our triangulation. We multiply them by their sign to obtain their absolut value inside the given maximal cone, and store it.

In [10]:
determinants = []
for simplex ∈ Q_vertices
    M = S[simplex[1][1] simplex[1][2] simplex[1][3]; simplex[2][1] simplex[2][2] simplex[2][3]; x y z]
    det_M = det(M)
    if det != 0
        sgn = sign(evaluate(det_M,[u_sample[1],u_sample[2],u_sample[3]]))
    end
    push!(determinants,sgn*det_M)
end
determinants

4-element Vector{Any}:
 (-4//3*x^2 - 4//3*y^2 - 4//3*z^2)//(x + z)
 (-4//3*x^2 - 4//3*y^2 - 4//3*z^2)//(y + z)
 (-x^3 - x^2*y - x^2*z - x*y^2 - x*z^2 - y^3 - y^2*z - y*z^2 - z^3)//(x*y + x*z + y*z + z^2)
 (5//9*x^3 + 5//9*x^2*y - 5//3*x^2*z + 5//9*x*y^2 + 5//9*x*z^2 + 5//9*y^3 - 5//3*y^2*z + 5//9*y*z^2 - 5//3*z^3)//(x*y + x*z + y*z + z^2)

To conclude, we sum all the absolute values of the determinants multiplied by the factor $\frac{1}{2 \|x\|^2}$ which includes the factor coming from the determinant and the norm in the definition of $\rho$. In this way we obtain the expression of $\rho(x)$ for $x$ in the chosen maximal cone.

In [11]:
rho = sum(det for det ∈ determinants)//(2*(x^2+y^2+z^2))

(-8//9*x - 8//9*y - 8//3*z)//(x*y + x*z + y*z + z^2)

We can repeat the same computation for all maximal cones, in order to obtain the expression of $\rho(x)$ for all $x\in \mathbb{R}^3$.